In [125]:
#from urllib.request import urlopen
import json

import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

import matplotlib.pyplot as plt

import numpy as np

## Shapes

In [195]:
# Making geopandas dataframes for 3 Portugal parts
geodf = gpd.read_file('concelhos-shapefile/concelhos.shp', encoding="utf-8")

continente_df = geodf.copy()
continente_df = continente_df[['NAME_1', 'NAME_2', 'CCA_2', 'geometry']] # CCA_2 for unique ID
continente_df.rename(columns={'NAME_1':'distrito', 'NAME_2':'concelho'}, inplace=True)

continente_df['concelho'] = continente_df['concelho'].str.capitalize()

madeira_df = continente_df[continente_df.distrito == 'Madeira']
continente_df = continente_df[continente_df.distrito != 'Madeira']

azores_df = continente_df[continente_df.distrito == 'Azores']
continente_df = continente_df[continente_df.distrito != 'Azores']

continente_df.head(2)

,distrito,concelho,CCA_2,geometry
0,Évora,Évora,0705,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506..."
1,Évora,Alandroal,0701,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343..."


In [196]:
# Simplification of Polygons for better performance
def simplify_region(polygon):
    tolerance=0.01
    if polygon.geom_type == 'Polygon':
        p = polygon.simplify(tolerance)      
    else:
        p = []
        for poly in polygon: # for multipolygon
            p_ = poly.simplify(tolerance) 
            p.append(p_)
        p = MultiPolygon(p)
    return p

continente_df.geometry = continente_df.geometry.apply(simplify_region)
madeira_df.geometry = madeira_df.geometry.apply(simplify_region)
azores_df.geometry = azores_df.geometry.apply(simplify_region)

In [198]:
# Creation of geojsons
continente_df.to_file("continente.geojson", driver = "GeoJSON", encoding="utf-8")
madeira_df.to_file("madeira.geojson", driver = "GeoJSON", encoding="utf-8")
azores_df.to_file("azores.geojson", driver = "GeoJSON", encoding="utf-8")

In [199]:
with open('continente.geojson', encoding='utf-8') as file:
    continente = json.loads(file.read())
    
with open('madeira.geojson', encoding='utf-8') as file:
    madeira = json.loads(file.read())

with open('azores.geojson', encoding='utf-8') as file:
    azores = json.loads(file.read())

In [200]:
continente['features'][0]['properties']

{'distrito': 'Évora', 'concelho': 'Évora', 'CCA_2': '0705'}

In [201]:
# add IDs to regions (features)
for feature in continente["features"]:
    feature['id'] = feature['properties']['CCA_2']
for feature in madeira["features"]:
    feature['id'] = feature['properties']['CCA_2']
for feature in azores["features"]:
    feature['id'] = feature['properties']['CCA_2']

In [202]:
azores['features'][0]['properties']

{'distrito': 'Azores', 'concelho': 'Angra do heroísmo', 'CCA_2': '4301'}

## Data

In [212]:
# generate data
data_list = []

for region in (continente, madeira, azores):
    np.random.seed(len(region['features']))
    
    ids = [i['properties']['CCA_2'] for i in region['features']]
    names = [i['properties']['concelho'] for i in region['features']]
    values = [int(100*np.random.random()) for i in range(len(region['features']))]
    
    data = pd.DataFrame([ids, names, values]).T.rename(columns={0:'id', 1: 'concelho', 2: 'value'})
    data['value'] = data['value'].astype(int)
    data_list.append(data)
    
continente_data, madeira_data, azores_data = data_list

In [204]:
palette = {'cutoff' : 'rgba(227, 78, 38, 1)', 
           'wave1' : 'rgba(247, 141, 31, 0.7)', 
           'wave2' : 'rgba(227, 78, 38, 1)',
           'scale0' : 'rgba(172, 185, 54, 0)',
           'scale1' : 'rgba(0, 120, 128, 1)',
           'scale2' : 'rgba(0, 102, 102, 1)',
           'scale3' : 'rgba(0, 70, 70, 1)',
           'borders' : 'rgba(0, 0, 0, 1)',
           'text' : 'rgba(114, 114, 114, 1)',
           'legendtext' : 'black',
           'bartext': 'black',
           'bartitletext' : 'black',
           'bubbletext' : 'black'
         }

In [218]:
fig = go.Figure()    

## Choropleth map ######
fig.add_choropleth(geojson=continente, locations=continente_data.id,
                        z=continente_data.value,
                        colorscale="teal",
                        zmin=0,
                        zmax=100,
                        hovertext = continente_data.concelho,
                        hoverinfo="text",
                        colorbar=dict(title={'text':'',
                                             'font':{#'size':24,
                                                     'family' : 'Arial',
                                                     'color' : palette['text']},
                                             'side':'right'},
                                      tickfont={#'size' : 20, 
                                                'color' : palette['bartext']},
                                      #ticks='outside',
                                      #len=0.1,
                                      #x=0.1
                                     ),
                        marker = dict(line=dict(width=1))
                        )
fig.update_geos(fitbounds="locations", bgcolor='rgba(0,0,0,0)', visible=False)
fig.show(renderer='browser')
#pio.write_image(fig, 'test.png', width=1600, height=800, engine='kaleido', scale=2)